In [43]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import ttest_rel

# 1) Strictly discover CSVs under T-Test/
file_paths = glob.glob('T-Test/**/*.csv', recursive=True)
print(f"🔍 Found {len(file_paths)} CSV files in 'T-Test/'\n")

# 2) Prepare metrics & classes
metrics = ['rmse', 'mae', 'r2']
classes = ['pts', 'reb', 'ast']

# 3) Containers for results and missing-fold tracking
metric_series = {m: {} for m in metrics}
missing_fold = []

# 4) Loop through each CSV
for path in file_paths:
    df = pd.read_csv(path)
    df.columns = [c.lower().strip() for c in df.columns]
    
    # 4a) Check presence of 'fold'
    if 'fold' not in df.columns:
        print(f"⚠️  Missing 'fold' column in: {path}")
        missing_fold.append(path)
        continue
    
    # 4b) Normalize perf‐columns to "<class>_<metric>"
    rename_map = {}
    for col in df.columns:
        if col == 'fold': 
            continue
        found_m = next((m for m in metrics if m in col), None)
        found_c = next((c for c in classes if c in col), None)
        if found_m and found_c:
            rename_map[col] = f"{found_c}_{found_m}"
    df = df.rename(columns=rename_map)
    
    # 4c) Split hyperparameter columns vs. perf columns
    perf_cols = {f"{c}_{m}" for c in classes for m in metrics}
    hyper_cols = [c for c in df.columns if c not in perf_cols and c != 'fold']
    base_name = os.path.splitext(os.path.basename(path))[0]
    
    # 4d) Group by hyperparameters (or treat whole file as one if none)
    groups = df.groupby(hyper_cols) if hyper_cols else [((), df)]
    for combo_vals, g in groups:
        # Build a unique model name
        if hyper_cols:
            combo = base_name + "_" + "_".join(f"{k}{v}" for k,v in zip(hyper_cols, combo_vals))
        else:
            combo = base_name
        
        # 4e) Sort on 'fold', warn if missing
        try:
            g = g.sort_values('fold')
        except KeyError:
            print(f"⚠️  Couldn't sort by 'fold' in {path} (combo={combo})")
            continue
        
        # 4f) For each metric, average across classes and store the 10‐element series
        for m in metrics:
            cols = [f"{c}_{m}" for c in classes]
            metric_series[m][combo] = g[cols].mean(axis=1).values

# 5) Report any completely skipped files
if missing_fold:
    print("\n🚩 The following files were skipped (no 'fold' column):")
    for f in missing_fold:
        print("  •", f)

# 6) Build summary tables & run paired t-tests
for m in metrics:
    dfm = pd.DataFrame(metric_series[m])
    if dfm.empty:
        print(f"\nℹ️  No data found for metric '{m}', skipping.")
        continue
    
    # decide sort (lower is better for rmse/mae; higher is better for r2)
    ascending = m in ['rmse', 'mae']
    means = dfm.mean().sort_values(ascending=ascending)
    best = means.index[0]
    
    # assemble summary
    summary = pd.DataFrame({
        'model': means.index,
        f'mean_{m}': means.values
    })
    
    # paired t-test vs. best
    pvals = []
    for mod in summary['model']:
        if mod == best:
            pvals.append(np.nan)
        else:
            _, p = ttest_rel(dfm[best], dfm[mod])
            pvals.append(p)
    summary['p_value_vs_best'] = pvals
    
    print(f"\n### {m.upper()} (best = {best})\n")
    display(summary)
groups = df.groupby(hyper_cols) if hyper_cols else [((), df)]
for combo_vals, g in groups:

    # --- NEW: normalise singleton group keys -------------------
    if not isinstance(combo_vals, tuple):
        combo_vals = (combo_vals,)
    # -----------------------------------------------------------

    model_key = raw_base.replace('_metrics', '').replace('_', ' ').title()
    if hyper_cols:
        params_str = ", ".join(
            f"{k.replace('_', ' ').title()}={v}"
            for k, v in zip(hyper_cols, combo_vals)
        )
        combo = f"{model_key} ({params_str})"
    else:
        combo = model_key


🔍 Found 12 CSV files in 'T-Test/'


### RMSE (best = elastic_alpha0.1_l1_ratio0.5)



model  mean_rmse  p_value_vs_best
0          elastic_alpha0.1_l1_ratio0.5   3.942111              NaN
1          elastic_alpha0.1_l1_ratio0.9   3.943392     5.995500e-01
2                        lasso_alpha0.1   3.943989     5.733017e-01
3          elastic_alpha0.1_l1_ratio0.1   3.952244     1.934060e-01
4                       ridge_alpha10.0   3.953467     4.373664e-01
..                                  ...        ...              ...
58  cnn_filters32_kernel_size3_epochs10   5.083508     2.775032e-10
59  cnn_filters16_kernel_size3_epochs10   5.098982     1.851462e-11
60  cnn_filters16_kernel_size5_epochs20   5.401336     7.024498e-04
61  cnn_filters32_kernel_size5_epochs10   5.436395     1.338633e-03
62  cnn_filters16_kernel_size5_epochs10   5.437348     1.333807e-03

[63 rows x 3 columns]


### MAE (best = elastic_alpha0.1_l1_ratio0.5)



model  mean_mae  p_value_vs_best
0          elastic_alpha0.1_l1_ratio0.5  3.062747              NaN
1          elastic_alpha0.1_l1_ratio0.9  3.063747     6.748601e-01
2                        lasso_alpha0.1  3.064491     5.835153e-01
3                       ridge_alpha10.0  3.068388     5.953106e-01
4                        ridge_alpha1.0  3.068573     5.891672e-01
..                                  ...       ...              ...
58  cnn_filters32_kernel_size3_epochs20  4.003219     1.143535e-09
59  cnn_filters16_kernel_size3_epochs10  4.004428     1.483249e-10
60  cnn_filters16_kernel_size5_epochs20  4.263607     6.799062e-04
61  cnn_filters32_kernel_size5_epochs10  4.301056     1.646142e-03
62  cnn_filters16_kernel_size5_epochs10  4.303658     1.614147e-03

[63 rows x 3 columns]


### R2 (best = elastic_alpha0.1_l1_ratio0.5)



model   mean_r2  p_value_vs_best
0          elastic_alpha0.1_l1_ratio0.5  0.408833              NaN
1          elastic_alpha0.1_l1_ratio0.9  0.407931     4.597294e-01
2                        lasso_alpha0.1  0.407562     4.551739e-01
3          elastic_alpha0.1_l1_ratio0.1  0.404731     2.418530e-01
4                       ridge_alpha10.0  0.403376     4.235882e-01
..                                  ...       ...              ...
58  cnn_filters32_kernel_size3_epochs10 -0.002046     1.335179e-11
59  cnn_filters16_kernel_size3_epochs10 -0.006021     9.507297e-13
60  cnn_filters16_kernel_size5_epochs20 -0.139875     2.235140e-03
61  cnn_filters32_kernel_size5_epochs10 -0.165660     5.167911e-03
62  cnn_filters16_kernel_size5_epochs10 -0.166000     5.147544e-03

[63 rows x 3 columns]

In [45]:
# %%  Model‑level summary  ────────────────────────────────────────────────
# Helper: strip the "(…)" param section, if present
import re, numpy as np

def base_model(name: str) -> str:
    m = re.match(r'^(.*?)\s*\(', name)   # everything before the first " ("
    return m.group(1).strip() if m else name.strip()

# 1) Gather per‑model lists of class‑metric values ------------------------
agg = {}
for combo, cm in class_metric_means.items():
    root = base_model(combo)
    agg.setdefault(root, {f'{c}_{m}': [] for c in classes for m in metrics})
    for k, v in cm.items():
        agg[root][k].append(v)

# 2) Build the table ------------------------------------------------------
rows = []
for model, d in agg.items():
    row = {'Model': model}
    # RMSE block
    row['AVG PTS RMSE'] = np.mean(d['pts_rmse'])
    row['AVG REB RMSE'] = np.mean(d['reb_rmse'])
    row['AVG AST RMSE'] = np.mean(d['ast_rmse'])
    # MAE block
    row['AVG PTS MAE']  = np.mean(d['pts_mae'])
    row['AVG REB MAE']  = np.mean(d['reb_mae'])
    row['AVG AST MAE']  = np.mean(d['ast_mae'])
    # R² block
    row['AVG PTS R²']   = np.mean(d['pts_r2'])
    row['AVG REB R²']   = np.mean(d['reb_r2'])
    row['AVG AST R²']   = np.mean(d['ast_r2'])
    rows.append(row)

df_model = pd.DataFrame(rows)

# Optional: choose a tidy column order
ordered_cols = [
    'Model',
    'AVG PTS RMSE', 'AVG REB RMSE', 'AVG AST RMSE',
    'AVG PTS MAE',  'AVG REB MAE',  'AVG AST MAE',
    'AVG PTS R²',   'AVG REB R²',   'AVG AST R²'
]
df_model = df_model[ordered_cols]

print("\n## Model‑level aggregate table")
display(df_model.style.format(precision=3))

# 3) 💾  Save to the same output folder
out_path = OUT_DIR / "model_level_stats.csv"
df_model.to_csv(out_path, index=False)
print(f"📑 CSV written to: {out_path.resolve()}")



## Model‑level aggregate table


📑 CSV written to: /Users/sshan/Documents/GitHub/NBA_Player_Props/summary_tables/model_level_stats.csv
